<a href="https://colab.research.google.com/github/WAT-ai/stock-forecasting-trading/blob/master/Stock_Trading_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Packages
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files

In [ ]:
# GLoVE embeddings
glove = torchtext.vocab.GloVe(name="42B", dim=300, max_vectors=50000)

In [ ]:
fast_text = torchtext.vocab.FastText()

In [ ]:
!pip install kaggle

In [ ]:
files.upload()

{}

In [ ]:
!mkdir ../root/.kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

Upload kaggle.json to root/.kaggle

In [ ]:
!kaggle datasets download -d yash612/stockmarket-sentiment-dataset

stockmarket-sentiment-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!kaggle datasets download -d ankurzing/sentiment-analysis-for-financial-news

sentiment-analysis-for-financial-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip stockmarket-sentiment-dataset.zip

Archive:  stockmarket-sentiment-dataset.zip
replace stock_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip sentiment-analysis-for-financial-news

Archive:  sentiment-analysis-for-financial-news.zip
replace FinancialPhraseBank/License.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def get_data():
    return csv.reader(open("stock_data.csv", "rt", encoding="latin-1"))

In [ ]:
for i, line in enumerate(get_data()):
    if i > 10:
        break
    print(line)

['Text', 'Sentiment']
['Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ  trade method 1 or method 2, see prev posts', '1']
['user: AAP MOVIE. 55% return for the FEA/GEED indicator just 15 trades for the year.  AWESOME.  ', '1']
["user I'd be afraid to short AMZN - they are looking like a near-monopoly in eBooks and infrastructure-as-a-service", '1']
['MNTA Over 12.00  ', '1']
['OI  Over 21.37  ', '1']
['PGNX  Over 3.04  ', '1']
['AAP - user if so then the current downtrend will break. Otherwise just a short-term correction in med-term downtrend.', '-1']
["Monday's relative weakness. NYX WIN TIE TAP ICE INT BMC AON C CHK BIIB  ", '-1']
['GOOG - ower trend line channel test & volume support.   ', '1']
['AAP will watch tomorrow for ONG entry.', '1']


In [ ]:
df = pd.read_csv("stock_data.csv")

In [ ]:
df.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


In [ ]:
df.size

11582

In [ ]:
# Split tweet into distinct words
def split_tweet(tweet):
    tweet = tweet.replace(".", " . ") \
                 .replace(",", " , ") \
                 .replace(";", " ; ") \
                 .replace("?", " ? ")
    return tweet.split()

In [ ]:
def create_glove_embedding(tweet, glove_vector):
  return sum(glove_vector[w] for w in split_tweet(tweet))

In [ ]:
def create_fast_text_embedding(tweet, fast_text_vector):
  return sum(fast_text_vector[w] for w in split_tweet(tweet))

In [ ]:
def compute_tweet_glove_vectors(glove_vector):
  train, test, valid = [], [], []
  for i, line in enumerate(get_data()):
    if i == 0:
      continue
    tweet = line[0]
    if i % 59 == 0:
      vector_sum = create_glove_embedding(tweet, glove_vector)
      label = torch.tensor(int(line[-1]) if int(line[-1]) == 1 else 0 ).long()
      if i % 5 < 3:
        train.append((vector_sum, label))
      elif i % 5 == 3:
        test.append((vector_sum, label))
      else:
        valid.append((vector_sum, label))

  return train, test, valid

In [ ]:
# def compute_tweet_fast_text_vectors(fast_text_vector):
#   train, test, valid = [], [], []
#   for i, line in enumerate(get_data()):
#     if i == 0:
#       continue
#     tweet = line[0]
#     if i % 59 == 0:
#       vector_sum = create_fast_text_embedding(tweet, fast_text_vector)
#       label = torch.tensor(int(line[-1]) if int(line[-1]) == 1 else 0 ).long()
#       if i % 5 <= 3:
#         train.append((vector_sum, label))
#       # elif i % 5 == 3:
#       #   test.append((vector_sum, label))
#       else:
#         valid.append((vector_sum, label))

#   return train, test, valid

In [ ]:
train, test, valid = compute_tweet_glove_vectors(glove)

In [ ]:
# train, test, valid = compute_tweet_fast_text_vectors(fast_text)

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=128, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=True)

In [ ]:
def get_accuracy(model, data_loader):
    correct, total = 0, 0
    for tweets, labels in data_loader:
        output = model(tweets)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return correct / total

In [ ]:
def train_nn(model, train_loader, valid_loader, num_epochs=5, learning_rate=1e-5):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  losses, train_accuracy, val_accuracy = [], [], []
  epochs = []
  for epoch in range(num_epochs):
    print(epoch)
    for tweets, labels in train_loader:
        optimizer.zero_grad()
        pred = model(tweets)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
    losses.append(float(loss))

    if epoch % 5 == 4:
        epochs.append(epoch)
        train_accuracy.append(get_accuracy(model, train_loader))
        val_accuracy.append(get_accuracy(model, valid_loader))
        print("Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (
            epoch+1, loss, train_accuracy[-1], val_accuracy[-1]))

    # plotting
    # plt.title("Training Curve")
    # plt.plot(losses, label="Train")
    # plt.xlabel("Epoch")
    # plt.ylabel("Loss")
    # plt.show()

    # plt.title("Training Curve")
    # plt.plot(epochs, train_accuracy, label="Train")
    # plt.plot(epochs, val_accuracy, label="Validation")
    # plt.xlabel("Epoch")
    # plt.ylabel("Accuracy")
    # plt.legend(loc='best')
    # plt.show()


In [ ]:
mymodel = nn.Sequential(nn.Linear(300, 200),
                        nn.ReLU(),
                        nn.Linear(200, 100),
                        nn.ReLU(),
                        nn.Linear(100, 80),
                        nn.ReLU(),
                        # nn.Linear(80, 50),
                        # nn.ReLU(),
                        nn.Linear(80, 40),
                        nn.ReLU(),
                        nn.Linear(40, 20),
                        nn.ReLU(),
                        nn.Linear(20, 2),
                        nn.Softmax(dim=0))
train_nn(mymodel, train_loader, valid_loader, num_epochs=100 , learning_rate=1e-4)
# get_accuracy(mymodel, test_loader)

0
1
2
3
4
Epoch 5; Loss 0.693091; Train Acc 0.724138; Val Acc 0.600000
5
6
7
8
9
Epoch 10; Loss 0.693031; Train Acc 0.655172; Val Acc 0.600000
10
11
12
13
14
Epoch 15; Loss 0.692961; Train Acc 0.689655; Val Acc 0.550000
15
16
17
18
19
Epoch 20; Loss 0.692881; Train Acc 0.741379; Val Acc 0.550000
20
21
22
23
24
Epoch 25; Loss 0.692779; Train Acc 0.758621; Val Acc 0.600000
25
26
27
28
29
Epoch 30; Loss 0.692649; Train Acc 0.793103; Val Acc 0.650000
30
31
32
33
34
Epoch 35; Loss 0.692491; Train Acc 0.793103; Val Acc 0.650000
35
36
37
38
39
Epoch 40; Loss 0.692296; Train Acc 0.793103; Val Acc 0.650000
40
41
42
43
44
Epoch 45; Loss 0.692055; Train Acc 0.793103; Val Acc 0.650000
45
46
47
48
49
Epoch 50; Loss 0.691753; Train Acc 0.793103; Val Acc 0.650000
50
51
52
53
54
Epoch 55; Loss 0.691377; Train Acc 0.793103; Val Acc 0.650000
55
56
57
58
59
Epoch 60; Loss 0.690898; Train Acc 0.793103; Val Acc 0.650000
60
61
62
63
64
Epoch 65; Loss 0.690282; Train Acc 0.793103; Val Acc 0.650000
65
66
67
6

In [ ]:
mymodel.eval()

Sequential(
  (0): Linear(in_features=300, out_features=200, bias=True)
  (1): ReLU()
  (2): Linear(in_features=200, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=80, bias=True)
  (5): ReLU()
  (6): Linear(in_features=80, out_features=40, bias=True)
  (7): ReLU()
  (8): Linear(in_features=40, out_features=20, bias=True)
  (9): ReLU()
  (10): Linear(in_features=20, out_features=2, bias=True)
  (11): Softmax(dim=0)
)

In [ ]:
def get_sentiment(query):
  mymodel.eval()
  embedding = create_embedding(query, glove)
  out = mymodel(embedding)
  class_prob, topclass = torch.max(out, dim=0)

  return topclass.item(), class_prob.item()


In [ ]:
 tweet = input()

In [ ]:
get_sentiment(tweet)